# Karhunen-Loève expansion

One may decompose certain stochastic processes into a bi-orthogonal series of uncorrelated and variance-ordered random variables. Such a **Karhunen-Loève expansion** (KLE) provides insights and lends itself to dimension reduction and simulation. Its discrete variant is demonstrated in the cells below. It is analogous to a **population PCA**. Note this representation is not specific to Gaussian processes in particular. It relates square-integrable stochastic processes in general.

In [ ]:
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.distributions as dist

from utils.kernels import SquaredExponential

In [ ]:
torch.set_default_dtype(torch.float64)

## Covariance matrix

In [ ]:
lower = -10
upper = 10
num_coords = 101

coords = torch.linspace(lower, upper, num_coords)

In [ ]:
sigma = 1 # standard deviation
tau = 1 # lengthscale parameter

kernel_rbf = SquaredExponential(sigma=sigma, tau=tau)
cov_rbf = kernel_rbf(coords, coords)

## Eigenvalues/vectors

In [ ]:
eigenvalues, eigenvectors = torch.linalg.eigh(cov_rbf) # ascending order

descending_ids = torch.argsort(eigenvalues, descending=True) # descending order
eigenvalues = eigenvalues[descending_ids]
eigenvectors = eigenvectors[:,descending_ids]

explained_var = torch.cumsum(eigenvalues, dim=0) / torch.sum(eigenvalues)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 3))

axes[0].plot(eigenvalues.numpy(), marker='.', color=plt.cm.Dark2(4), alpha=0.7, clip_on=False)
axes[0].set(xlabel='component', ylabel='eigenvalue')
axes[0].set_xlim((0, len(eigenvalues)-1))
axes[0].set_ylim((0, eigenvalues.max()))

axes[1].plot(explained_var.numpy(), marker='.', color=plt.cm.Dark2(4), alpha=0.7, clip_on=False)
axes[1].set(xlabel='no. components', ylabel='explained var. ratio')
axes[1].set_xlim((0, len(explained_var)-1))
axes[1].set_ylim((0, 1))

for ax in axes.ravel():
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(coords.numpy(), eigenvectors[:,:5].numpy(), alpha=0.7)
ax.set(xlabel='coordinate', ylabel='value')
ax.set_xlim((coords[0], coords[-1]))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

## Simulation

In [ ]:
num_components = 25

retained_eigenvalues = eigenvalues[:num_components]
retained_eigenvectors = eigenvectors[:,:num_components]

In [ ]:
num_samples = 10

normal_samples = dist.Normal(
    loc=torch.zeros_like(retained_eigenvalues),
    scale=torch.sqrt(retained_eigenvalues)
).sample((num_samples,))

samples = normal_samples @ retained_eigenvectors.T

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(coords.numpy(), samples.T.numpy(), alpha=0.7)
ax.set(xlabel='coordinate', ylabel='value')
ax.set_xlim((coords[0], coords[-1]))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()